Restrições hard:
Não podemos alocar uma turma numa sala com quantidade de assentos menor
que o número de alunos matriculados.
Não podemos alocar uma turma que necessita de acessibilidade numa sala que
não é acessível
Restrições soft
• Devemos minimizar a taxa de ocupação de salas: (número de alunos
– tamanho da sala)/tamanho da sala.
• Restrição de qualidade da sala: Uma turma de qualidade baixa pode
ser alocada numa sala de qualidade alta, mas o contrário não é
verdade.
• Restrição de salas no mesmo horário: penalizar uma distribuição
onde uma mesma turma esteja numa sala diferentes.
• Restrição de calouros na mesma sala: penalizar distribuições onde
turmas de calouros de um mesmo curso estejam em salas
diferentes.

In [438]:
#definitions

import csv
#leitura andre
#df = csv.DictReader(open("cenario1-salas.csv")) 
#df1 = csv.DictReader(open("cenario1-turmas.csv",encoding='utf-8'))

#leitura paulo
df = csv.DictReader(open(r"C:\Users\Paulo\Desktop\TrabalhoSAD\cenario1-salas.csv",encoding='utf-8'))
df1 = csv.DictReader(open(r"C:\Users\Paulo\Desktop\TrabalhoSAD\cenario1-turmas.csv",encoding='utf-8'))
turmas = {}
salas = {}
horarios = {}
id_Salas = []
horario = {}
x = 0

In [439]:
#transforma o horario para um formato mais amigavel
def transHorario(horario):
    list = []
    turmaTemp = horario.split("-")
    for i in range(len(turmaTemp)):
        dia = int(turmaTemp[i][0])
        turno = turmaTemp[i][1]
        horarioIni = int(turmaTemp[i][2])

        if(turno == 'M'):
            horarioIni = 1 + horarioIni/2 
        if(turno == 'T'):
            horarioIni = 4 + horarioIni/2 

        elif(turno == 'N'):
            horarioIni = 7 + horarioIni/2
        
        if(int(turmaTemp[i][0]) == 2):
            dia = 'seg'
        elif(int(turmaTemp[i][0]) == 3):
            dia = 'ter'
        elif(int(turmaTemp[i][0]) == 4):
            dia = 'qua'
        elif(int(turmaTemp[i][0]) == 5):
            dia = 'qui'
        elif(int(turmaTemp[i][0]) == 6):
            dia = 'sex'
        else:
            dia = 'sab'
        list.append((dia,int(horarioIni)))
    return list

In [440]:
    
for row in df1:
    turmas[x] = {"disciplina": row['disciplina'],"prof": row['professor'], "horario": transHorario(row['dias_horario']),
                 "alunos": int(row['numero_alunos']), "curso": row['curso'], 
                 "periodo": int(row['período']), "acess": int(row["acessibilidade"]), "quali": int(row["qualidade"])}
    x+=1
for row in df:
    salas[row['id_sala']] = {"cad": int(row['numero_cadeiras']), "acess": int(row['acessivel']), "quali": int(row['qualidade'])}
    id_Salas.append(row['id_sala'])

for sala in id_Salas:
    horarios[sala] =  { "seg":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "ter":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "qua":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "qui":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "sex":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 },
                        "sab":{1: 0, 2: 0, 3: 0, 4: 0, 5: 0 , 6: 0 , 7: 0 , 8: 0 }}




In [441]:

sorted_salas = sorted(salas.items(), key=lambda sala: (sala[1]['acess'],sala[1]['cad']), reverse=True)

sorted_turmas = sorted(turmas.items(), key=lambda turma: (turma[1]['acess'], turma[1]['alunos']), reverse=True)


In [442]:
def achaSalasDisponiveis(horarios, turma, salas):
    h = turma['horario']
    #recebe a lista de horarios dessa turma
    salasDisponiveis = []
    #lista que recebe a solução
    for sala in salas:
        
        for j in range(len(horarios)):
            contador = 0
            for k in range(len(h)):
                dia = h[k][0]
                #recebe o dia do horario k 
                horario = h[k][1] 
                #recebe a hora do horario k
                if(horarios[sala[0]][dia][horario] == 0): 
                    contador += 1 
                    #se o horario estiver disponivel é incrementado este contador
        if(contador == len(h)): 
            #se o contador chegar ao mesmo tamanho da lista de horario, significa que essa 
            # sala está livre em todos os horarios dessa turma, logo ela pode ser alocada.
            salasDisponiveis.append(sala) #adiciona a sala avaliada na lista resultado
    return salasDisponiveis

In [443]:
def achaSalasPossiveis(horarios, turma, salas):
    salasDisponiveis = achaSalasDisponiveis(horarios,turma, salas) 
    #busca as salas que ainda estão livres
    salasPossiveis = []
    for sala in salasDisponiveis:
        # Verifica a acessibilidade e o tamanho da turma. Ex:
        # Acess = 0 retornar turmas com acess 0 e 1
        # Acess = 1 retornar turmas com acess 1
        if(sala[1]['cad'] >= turma['alunos'] and sala[1]['acess'] >= turma['acess']): 
            salasPossiveis.append(sala)
    return salasPossiveis

In [444]:

def alocaTurmasGulosa(horarios, turmas, salas):
    
    salasPossiveis = []
    
    for turma in turmas:
        
        h = turma[1]['horario']
        salasPossiveis = achaSalasPossiveis(horarios, turma[1], salas)
        
        if(salasPossiveis == []):
            print("deu conflito rs", turma[1]['alunos'])
        else:
            for i in range(len(h)):
                dia = h[i][0] 
                #recebe o dia do horario k 
                horario = h[i][1] 
                #recebe a hora do horario k
                sala = salasPossiveis[0][0]
                horarios[sala][dia][horario] = turma[0] #coloca o id(lista não ordenada) da turma no horario.
                
    return horarios
            

In [445]:
#calcula a taxa de ocupação de uma sala em relação a uma turma
def taxaOcupacao(turma, sala):
   return (sala['cad'] - turma['alunos'])/sala['cad']

#retorna 1 caso a sala esteja em uma sala de qualidade adequada
#e retorna 0 caso contrario
def taxaQualidade(turma, sala):
    if sala['quali'] > turma['quali']:
        return 0
    return 1

#retorna 1 caso a sala esteja em uma sala de acessibilidade adequada
#e retorna 0 caso contrario
def taxaAcessibilidade(turma, sala):
    if (sala['acess'] == turma['acess']):
        return 1
    return 0

In [446]:
#analisa qualidade da solução atual
def analisaQualidade(solucao, turmas, salas):
    somatorio = []
    for i in range(len(solucao)):
        # loop que passa pelas salas
        id_sala = solucao[i][0]
        for dia,value in solucao[i][1].items():
            # Loop que passa pelos dias e seus valores
            # dia  = seg, ter ... | Value = 1: idTurma, 2: 0 ...
            for horario, id_turma in value.items():
                # Verifica se teve aula no dia.
                # Caso sim analisa as taxas de qualidade para essa turma
                if id_turma > 0:
                    ocup = taxaOcupacao(turmas[id_turma], salas[id_sala])
                    quali = taxaQualidade(turmas[id_turma], salas[id_sala])
                    acess = taxaAcessibilidade(turmas[id_turma], salas[id_sala]) 
                    obj = {"ocup": ocup,"acess": acess ,"quali": quali}
                    somatorio.append(obj)
    return somatorio

def qualidadeDaSolucao(solucao, turmas, salas):
    #calcula a qualiade da solução no geral
    #quanto mais as taxas se aproximem de 1,
    # melhor é a qualidade da solução
    erros = analisaQualidade(solucaoGulosa, turmas, salas)
    sumOcup, sumQuali, sumAcess = (0,0,0)
    for erro in erros:
        sumOcup += erro['ocup']
        sumAcess += erro['acess']
        sumQuali += erro['quali']
    print("erro relativo da ocupação é:", sumOcup/len(erros))
    print("erro relativo da qualidade é:", sumQuali/len(erros))
    print("erro relativo da acessibilidade é:", sumAcess/len(erros))



In [447]:
def exibeSolucao(solucao, turmas, salas):
    # Escreve a Sala seguido do seu horario na semana
    # Somente os horarios com aulas
    # Avisa caso a sala não tenha aula no dia
    for i in range(len(solucao)):
        # loop que passa pelas salas
        print("Sala:", solucao[i][0])
        for key,value in solucao[i][1].items():
            # Loop que passa pelos dias e seus valores
            # Key = seg, ter ... | Value = 1: idTurma, 2: 0 ...
            teveAula = False
            print("{}:".format(key))
            for chave, valor in value.items():
                # Verifica se teve aula no dia.
                # Caso sim escreve o horario e a turma.
                if valor > 0:
                    teveAula = True
                    print("Horario:",chave, '-',turmas[valor]['disciplina'])
            if teveAula ==False:
                # Caso não escreve a sala e o dia que não teve.
                print("Não tem aula na sala {} para {}".format(solucao[i][0],key))
        if i != len(solucao)-1:
            print("-------------------------------------------------------------------")

In [448]:
solucaoGulosa =  list(alocaTurmasGulosa(horarios, sorted_turmas, sorted_salas).items())
qualidadeDaSolucao(solucaoGulosa, turmas, salas)
print()
exibeSolucao(solucaoGulosa, turmas, salas)

erro relativo da ocupação é: 0.5332565284178187
erro relativo da qualidade é: 0.7956989247311828
erro relativo da acessibilidade é: 0.24731182795698925

Sala: 101
seg:
Horario: 2 - COMP0197
Horario: 3 - COMP0281
Horario: 4 - COMP0396
Horario: 5 - COMP0197
Horario: 6 - COMP0196
Horario: 7 - COMP0217
Horario: 8 - COMP0347
ter:
Horario: 1 - COMP0212
Horario: 2 - COMP0219
Horario: 3 - COMP0203
Horario: 4 - COMP0219
Horario: 5 - COMP0279
Horario: 6 - COMP0337
Horario: 7 - COMP0211
Horario: 8 - COMP0284
qua:
Horario: 2 - COMP0197
Horario: 3 - COMP0219
Horario: 4 - COMP0396
Horario: 5 - COMP0197
Horario: 6 - COMP0203
Horario: 7 - COMP0217
Horario: 8 - COMP0347
qui:
Horario: 1 - COMP0212
Horario: 2 - COMP0203
Horario: 3 - COMP0281
Horario: 4 - COMP0219
Horario: 5 - COMP0279
Horario: 6 - COMP0337
Horario: 7 - COMP0211
Horario: 8 - COMP0284
sex:
Horario: 2 - COMP0197
Horario: 4 - COMP0396
Horario: 5 - COMP0197
Horario: 6 - COMP0203
Horario: 7 - COMP0211
Horario: 8 - COMP0347
sab:
Horario: 1 - CO